# ETL (Extrair, Transformar e Carregar) da camada Raw para Silver
Este notebook tem como objetivo aplicar, em nível moderado, a transformação e limpeza dos dados brutos da camada Raw para a camada Silver, a fim de fornecer uma “visão corporativa” da principal entidade do negócio. Ele processa os dados brutos do banco de dados da Comunicação de Acidente de Trabalho (CAT) do INSS.

# Extração
Seção de extração dos dados brutos do arquivo CSV



In [10]:
!pip install pyspark
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder \
    .appName("CSVtoPostgresETL") \
    .config("spark.jars", "path/to/postgresql-42.x.x.jar") \
    .getOrCreate()

df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/content/dados_brutos.csv")

df.show()
df.printSchema()


+--------------------------+--------------+--------------------+--------------------+------------------+----------------------------+-------------------+--------------------+-----------------+---------------------+--------------------+--------------------+---------------------------+--------------------+---------+----------------+--------------------+--------------------+---------------+-----------------------+---------------+---------------+----------------+-------------------+
|Agente  Causador  Acidente|Data Acidente1|                 CBO|              CID-10|CNAE2.0 Empregador|CNAE2.0 Empregador Descrição|       Emitente CAT|Espécie do benefício|Filiação Segurado|Indica Óbito Acidente|          Munic Empr|   Natureza da Lesão|Origem de Cadastramento CAT|Parte Corpo Atingida|     Sexo|Tipo do Acidente|UF  Munic.  Acidente|UF Munic. Empregador|Data Acidente18|Data Despacho Benefício|Data Acidente20|Data Nascimento|Data Emissão CAT|CNPJ/CEI Empregador|
+--------------------------+----

# Transformação
Nesta seção, iremos transformar e limpar os dados vindos da camada RAW para a camada Silver, ainda utilizando o Spark.

### Importações e configuração
Aqui iremos importar o necessário do pyspark, definir algumas variáveis para auxiliar nas transformações, e fazer uma pequena transformação dos dados não classificados para nulos, assim facilitando as próximos transformações.

In [12]:
from pyspark.sql.functions import col, count, when, sum as spark_sum
from pyspark.sql.types import StringType

nao_classif_string = '{ñ class}'
nao_classif_numeric = 0

limite_repeticao = 0.95
limite_nulos = 0.95

total_linhas = df.count()
colunas_a_remover = []
novas_colunas = []

for coluna in df.columns:
    tipo_coluna = df.schema[coluna].dataType

    # Fazemos isso pois algumas colunas possuem '.', o que buga
    coluna_escaped = col(f"`{coluna}`")

    if isinstance(tipo_coluna, StringType):
        expressao = when(coluna_escaped == nao_classif_string, None).otherwise(coluna_escaped).alias(coluna)
    else:
        expressao = when(coluna_escaped == nao_classif_numeric, None).otherwise(coluna_escaped).alias(coluna)

    novas_colunas.append(expressao)

# Executando as expressões de trocar dos valores não classificados para nulos
df_limpo = df.select(*novas_colunas)

### Remoção de Colunas com Dados Não Classificados ou Constante
As características utilizadas para as colunas caírem nesse termo são:

- 95%+ dos valores são iguais
- 60%+ dos valores são nulos/não classificados
- Variância = 0
- Não agrega informação

In [13]:
for coluna in df_limpo.columns:
    # Fazemos isso pois algumas colunas possuem '.', o que buga
    coluna_escaped = f"`{coluna}`"

    # 1. Verifica Porcentagem de Nulos
    # Como já retiramos os não classificados, agora só contar os nulos diretamente
    qtd_nulos = df_limpo.select(count(when(col(coluna_escaped).isNull(), 1))).collect()[0][0]

    porcentagem_nulos = (qtd_nulos / total_linhas) * 100

    if porcentagem_nulos > (limite_nulos * 100):
        print(f"Removendo {coluna}: {porcentagem_nulos:.2f}% de nulos (incluindo não classificados convertidos).")
        colunas_a_remover.append(coluna)
        continue

    # 2. Verifica valores repetitivos (>95%)
    repeticao_maxima = df_limpo.groupBy(coluna_escaped).count() \
        .agg({"count": "max"}).collect()[0][0]

    if repeticao_maxima is None: repeticao_maxima = 0

    porcentagem_maxima = (repeticao_maxima / total_linhas) * 100

    if porcentagem_maxima > (limite_repeticao * 100):
        print(f"Removendo {coluna}: {porcentagem_maxima:.2f}% de valor repetido.")
        colunas_a_remover.append(coluna)

if colunas_a_remover:
    df_final = df_limpo.drop(*colunas_a_remover)
    print(f"\nTotal removido: {len(colunas_a_remover)}")
else:
    df_final = df_limpo
    print("\nNenhuma coluna removida.")

Removendo Emitente CAT: 95.82% de valor repetido.
Removendo Espécie do benefício: 100.00% de valor repetido.
Removendo Filiação Segurado: 99.13% de valor repetido.
Removendo Indica Óbito Acidente: 99.54% de valor repetido.
Removendo Origem de Cadastramento CAT: 100.00% de valor repetido.
Removendo Data Despacho Benefício: 100.00% de valor repetido.
Removendo CNPJ/CEI Empregador: 96.25% de nulos (incluindo não classificados convertidos).

Total removido: 7


### Remoção de Colunas Altamente Correlacionada
Serão identificadas e removidas colunas que possuem conteúdo idêntico ou derivado direto de outras colunas existentes, eliminando redundâncias desnecessárias.

In [14]:
import re

def tirar_numero(coluna):
  return re.sub(r'\d+', '', coluna)

colunas_a_remover = []
colunas_limpas = set()

for coluna in df.columns:
  nome_limpo = tirar_numero(coluna).lower()
  print(nome_limpo)

  if nome_limpo in colunas_limpas:
    print("Removendo coluna: ", nome_limpo)
    colunas_a_remover.append(coluna)
  else:
    colunas_limpas.add(nome_limpo)


df_final = df_final.drop(*colunas_a_remover)
df_final.show()



agente  causador  acidente
data acidente
cbo
cid-
cnae. empregador
cnae. empregador descrição
emitente cat
espécie do benefício
filiação segurado
indica óbito acidente
munic empr
natureza da lesão
origem de cadastramento cat
parte corpo atingida
sexo
tipo do acidente
uf  munic.  acidente
uf munic. empregador
data acidente
Removendo coluna:  data acidente
data despacho benefício
data acidente
Removendo coluna:  data acidente
data nascimento
data emissão cat
cnpj/cei empregador
+--------------------------+--------------+--------------------+--------------------+------------------+----------------------------+--------------------+--------------------+--------------------+---------+----------------+--------------------+--------------------+---------------+----------------+
|Agente  Causador  Acidente|Data Acidente1|                 CBO|              CID-10|CNAE2.0 Empregador|CNAE2.0 Empregador Descrição|          Munic Empr|   Natureza da Lesão|Parte Corpo Atingida|     Sexo|Tipo do Aciden

## Tratando/Limpando valores nulos

O tratamento dos campos nulos foi definido com base no impacto analítico de cada variável:

(Agente Causador, CBO e UF. Munic. do Acidente): Valores nulos serão alterados para o termo "Não identificado". Isso preserva o registro para contagem total, já que outras variáveis permitem análises parciais

(CID e CNAE): São campos essenciais. Sem o CID, perde-se a causa médica; sem o CNAE, inviabiliza-se a análise por setor econômicos. Se nulos, serão descartados, pois a falta desses dados inviabiliza a análise.


In [15]:
cols_para_preencher = ["Agente  Causador  Acidente", "CBO", "`UF  Munic.  Acidente`"]
cols_criticas_para_drop = ["CID-10", "`CNAE2.0 Empregador`", "`CNAE2.0 Empregador Descrição`"]

df_tratado = df_final \
    .na.fill("Não identificado", subset=cols_para_preencher) \
    .na.drop(subset=cols_criticas_para_drop)

print(f"Total antes: {df_final.count()}")
print(f"Total depois: {df_tratado.count()}")

df_tratado.show()

Total antes: 157845
Total depois: 145692
+--------------------------+--------------+--------------------+--------------------+------------------+----------------------------+--------------------+--------------------+--------------------+---------+----------------+--------------------+--------------------+---------------+----------------+
|Agente  Causador  Acidente|Data Acidente1|                 CBO|              CID-10|CNAE2.0 Empregador|CNAE2.0 Empregador Descrição|          Munic Empr|   Natureza da Lesão|Parte Corpo Atingida|     Sexo|Tipo do Acidente|UF  Munic.  Acidente|UF Munic. Empregador|Data Nascimento|Data Emissão CAT|
+--------------------------+--------------+--------------------+--------------------+------------------+----------------------------+--------------------+--------------------+--------------------+---------+----------------+--------------------+--------------------+---------------+----------------+
|      Rua e Estrada - S...|       2023/03|252105-Administrado

# Carregação